In [11]:
import numpy as np
import pandas as pd
import requests
import json
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import time
import math
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.tile_providers import ESRI_IMAGERY, get_provider

In [14]:
with open('variables/city_centers.json') as f_in:
    city_centers = json.load(f_in)

sf_lat = city_centers['sf'][0]
sf_lon = city_centers['sf'][1]
chicago_lat = city_centers['chicago'][0]
chicago_lon = city_centers['chicago'][1]
nyc_lat = city_centers['nyc'][0]
nyc_lon = city_centers['nyc'][1]

In [2]:
with open('json_data/sf_venues_results.json') as f_in: 
    sf_venues_results = json.load(f_in)
with open('json_data/chicago_venues_results.json') as f_in:
    chicago_venues_results = json.load(f_in)
with open('json_data/nyc_venues_results.json') as f_in:
    nyc_venues_results = json.load(f_in)

In [51]:
unique_ids = set()

sf_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(sf_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            sf_data['latitudes'].append(item['venue']['location']['lat'])
            sf_data['longitudes'].append(item['venue']['location']['lng'])
            sf_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(sf_data['latitudes'], sf_data['longitudes']):
    x, y = latlon2mercator(lat, lon)
    sf_data['x_coords'].append(x)
    sf_data['y_coords'].append(y)

In [40]:
unique_ids = set()

chicago_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(chicago_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            chicago_data['latitudes'].append(item['venue']['location']['lat'])
            chicago_data['longitudes'].append(item['venue']['location']['lng'])
            chicago_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(chicago_data['latitudes'], chicago_data['longitudes']):
    x, y = latlon2mercator(lat, lon)
    chicago_data['x_coords'].append(x)
    chicago_data['y_coords'].append(y)

In [49]:
tile_provider = get_provider(OSM)
sf_x, sf_y = latlon2mercator(sf_lat, sf_lon)
# p = figure(x_range = (sf_x - 100, sf_x + 100), y_range= (sf_y - 100, sf_y + 100),
#            x_axis_type = "mercator", y_axis_type = "mercator")
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.circle(x = 'x_coords', y = 'y_coords', source = sf_data)
p.add_tile(tile_provider)
show(p)

In [25]:
tile_provider = get_provider(ESRI_IMAGERY)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

show(p)

In [50]:
def latlon2mercator(lat, lon):
    r_earth = 6378137.000
    x = r_earth * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)